# 3.2 数据预处理

本节将介绍几种常见的数据预处理方法：缺省值补全、数据无量纲化、列表特征编码和数据离散化。其中，sklearn.impute模块为填补缺失值专用。sklearn.preprocessing模块提供了其他大多数常用的数据预处理方法。

## 3.2.1 缺失值补全

我们在处理实际数据时，往往会存在若干缺失值，通常需要从已有的数据中推断出缺失的数值，即插值（Imputation）操作。sklearn中提供了两种类型的插值算法——单变量插值与多变量插值算法。前者只使用第i个特征维度中的非缺失值来插补第i个特征维中的值；后者使用整个可用特征维度来估计缺失的值。

（1）单变量插值（SimpleImputer）

sklearn.impute模块SimpleImputer类提供了单变量算法处理缺失值的功能，在进行插值时，既可以使用指定的常数值替换缺失值，也可以使用缺失值所在的行或列中的统计数据，如平均值、中位数或者众数等，来替换缺失值。

使用SimpleImputer类生成Imputer类型对象时，使用参数missing_values指定真实数据中缺失值的标识，使用参数strategy指定插值策略，取值为"mean"、"median"、"most_frequent"或"constant"，分别表示使用每列的平均值、中位数、众数或指定某个常数来替换缺失值，该常数使用fill_value参数指定。
以下代码给出了使用常数10来替换编码为0的缺失值。


In [1]:
import numpy as np    
from sklearn.impute import SimpleImputer  # 导入包
#定义Imputer对象
i=SimpleImputer(missing_values=0,strategy='constant',fill_value=10) 
i.fit([[0,1,2]])  # fit()函数让Imputer对象适应(拟合)数据，参数为2维数组     
X = np.array([[0,1,2], [6,0,5], [7,8,0]])  # 原始数据,列数应与fit函数参数列数相同
X1 = i.transform(X)  # 将X中的所有缺失值填补，返回填补后的二维数组    
print(X1, type(X1))  

[[10  1  2]
 [ 6 10  5]
 [ 7  8 10]] <class 'numpy.ndarray'>


fit函数通过设置参数指明了要转换的二维数组的列数，SimpleImputer也提供了fit_transform函数，直接进行拟合数据并对缺省值填补，例如：

In [9]:
i2=SimpleImputer(missing_values=0,strategy='constant',fill_value=10) 
X = np.array([[0,1,2,0], [6,0,5,0], [7,8,0,0]])  # 原始数据,3行4列
X2 = i2.fit_transform(X)  # 先fit后进行transform操作，返回填补后的二维数组    
print(X2, type(X2))

[[10  1  2 10]
 [ 6 10  5 10]
 [ 7  8 10 10]] <class 'numpy.ndarray'>


以下代码分别使用列的平均值和众数来填补缺省值。

In [10]:
import pandas as pd
i3 = SimpleImputer(missing_values=np.nan, strategy='mean')
print( i3.fit_transform([[np.nan,1,3],[0,np.nan,2],[4,3,1]]))
i4 = SimpleImputer(missing_values='', strategy='most_frequent')
p=pd.DataFrame([['','k','x'],['a','m',''],['b','','y'],['a','m','x']],dtype="category")  # 定义DataFrame对象
print(i4.fit_transform(p))

[[2. 1. 3.]
 [0. 2. 2.]
 [4. 3. 1.]]
[['a' 'k' 'x']
 ['a' 'm' 'x']
 ['b' 'm' 'y']
 ['a' 'm' 'x']]


（2）多变量插值（IterativeImputer）

多变量插值IterativeImputer类利用各列非缺省值构建函数为每个缺失值进行建模。建模采用迭代循环方式执行，即在每个步骤中，将目标列指定为输出y，将其他列视为输入X，使用一个回归模型对(Ｘ，ｙ)进行拟合，然后使用这个回归模型来预测缺失的ｙ值。IterativeImputer类也是通过参数missing_values指定真实数据中缺失值的标识，通过参数max_iter控制迭代次数，默认值为10。

以下代码给出了使用多变量插值方法进行数据插补的示例。

In [11]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer    
imp = IterativeImputer(missing_values=np.nan)  
imp.fit([[1, 3],[2, 5],[3, 7]])  # 对每一列使用其他列进行回归  
X_test = [[np.nan, 11], [6, np.nan], [np.nan, 6],[3,17]]  # 原始数据
print(imp.transform(X_test))  # 打印插值后的数据

[[ 4.9999985  11.        ]
 [ 6.         12.999999  ]
 [ 2.49999975  6.        ]
 [ 3.         17.        ]]


## 3.2.2 数据无量纲化

许多机器学习算法中的目标函数是假设所有的特征为零均值且具有相同方差。如果数据某维特征的方差比其他维度的特征大几个数量级，那么这个特征可能会在学习算法中占据主导位置，导致模型无法从其他特征中学习到好的分类效果。因此，我们常常需要对数据进行去量纲的操作，提升模型的收敛速度和精度。

sklearn.preprocessing中提供了多种线性的或非线性的无量纲化相关函数，在这里只介绍几种常见的无量纲化方法。

（1）最小最大归一化（MinMaxScaler）

归一化方法将每列数据缩放到给定的最小值和最大值之间。转换公式为：
                   
$$X_{scaled}=(X-min⁡( X))\frac{Max-Min}{max⁡( X)-min⁡( X)}+Min$$

其中，Min和Max是指定的最小值和最大值。通常使用MinMaxScaler类进行归一化操作，其中一个重要的参数feature_range用于控制缩放的范围，默认值为[0,1]，示例如下。

In [13]:
from sklearn.preprocessing import MinMaxScaler  
X_train = [[1, -1, 2], [2, 0, 0], [0, 1, -1]]
min_max_scaler = MinMaxScaler(feature_range=[0, 1])  
X_train_minmax = min_max_scaler.fit_transform(X_train)    
print(X_train_minmax)

[[0.5        0.         1.        ]
 [1.         0.5        0.33333333]
 [0.         1.         0.        ]]


（2）最大绝对值归一化（MaxAbsScaler）

最大绝对值归一化与最小最大归一化类似，只不过它将每列数据按最大绝对值进行归一化。转换公式表达为：

$$X_{scaled}=\frac{X}{max⁡( abs (X))}$$

因此，每列数据都会映射在[-1,1]范围内，示例如下。

In [14]:
from sklearn.preprocessing import MaxAbsScaler  
X_tr = [[1, -1, 2], [2, 0, 0], [0, -2, -1]]
mas = MaxAbsScaler()  # 实例化MaxAbsScaler对象
X_re = mas.fit_transform(X_tr)  # 注意先进行fit，后进行transform
print(X_re)

[[ 0.5 -0.5  1. ]
 [ 1.   0.   0. ]
 [ 0.  -1.  -0.5]]


不难发现，每列数据都按该列的最大绝对值进行了归一化操作。需要注意的是，一旦进行了训练，即执行了fit操作，每列的最大绝对值就确定了，后续进行transform操作时，按训练时确定的最大绝对值进行归一化，此时归一化后的数据有可能不在[-1,1]取间。例如执行了上面的代码后，再执行如下操作，会发现预处理后的数据并不在[-1,1]区间。

In [15]:
print(mas.transform([[5, -5, 3]]))

[[ 2.5 -2.5  1.5]]


（3）Z-score标准化（StandardScaler）

Z-score标准化是指将每列数转换为服从为均值为0，方差为1的正态分布（即标准正态分布）的数据。设某列原始数据X的均值为μ，标准差为σ，则转换公式为：

$$X_{std}=\frac{X-μ}{σ}$$  

Z-score标准化是非常常见的归一化方法，示例如下：


In [16]:
from sklearn.preprocessing import StandardScaler as ss
scaler = ss()  # 实例化
X_train = [[1, -1, 2], [2, 0, 4], [3, 2, -6]]
scaler.fit(X_train)  # 在训练集上训练，得到均值和方差
print('mean', scaler.mean_)  # 查看得到的均值
print('std', scaler.scale_)  # 查看得到的方差
X_train_re = scaler.transform(X_train)  # 在训练集上归一化
print('X_train_re', X_re)  # 打印训练集归一化结果
X_test_re = scaler.transform([[1, 2, 3]])  # 在测试集上归一化
print('X_test_re', X_test_re)  # 打印测试集归一化结果

mean [2.         0.33333333 0.        ]
std [0.81649658 1.24721913 4.3204938 ]
X_train_re [[ 0.5 -0.5  1. ]
 [ 1.   0.   0. ]
 [ 0.  -1.  -0.5]]
X_test_re [[-1.22474487  1.33630621  0.69436507]]


## 3.2.3 类别特征编码

大多数机器学习算法只能够处理数值型数据，不能处理文字信息，然而实际应用中有很多数据都不是连续型的数值，而是类别型（categorical）的文字, 比如性别为“男”，“女”等。因此，在执行机器学习算法前，必须对类别型文本进行编码，将其转换为数值型。

Sklearn的preprocessing模块提供了多种对数据进行编码的类，下面列举一些常用的类及其使用方法。

（1）标签编码（LabelEncoder） 

对于分类任务，需要根据数据特征，预测对应的类别标签，而类别标签有可能是本文信息。preprocessing模块中的LabelEncoder类针对文本标签进行编码，从训练样本中获取标签，并将N个标签中的每一个标签映射到一个从0到N-1，从而得到每个标签的整数编码。通常，标签数据都是一维数组。示例如下：


In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(["beijing", "shanghai", "beijing", "shenzhen"])
print("labels: ", list(le.classes_))  # 打印所有的标签集合
print(le.transform(["beijing", "shanghai"]))  # 打印给定标签对应的编码
print(list(le.inverse_transform([2, 2, 1])))  # 打印编码对应的原始标签

labels:  ['beijing', 'shanghai', 'shenzhen']
[0 1]
['shenzhen', 'shenzhen', 'shanghai']


（2）多标签二值化编码（MultiLabelBinarizer）

对于多标签分类任务，样本的类别可能不止一个，例如将某个人生活过的城市作为标签，则标签可以有多个。这就需要采外的编码方式，其中最简单的就是多标签二值化编码。所谓多标签二值化编码，就是将各种标签组合统一考虑，设所有可能的标签取值数量为N，则将编码设置为长度为N的二进制向量，每个标签对应二进制的一位。在进行编码时，拥有该标签，对应的位设置位1，否则设置为0。

例如，设大城市共有7个，分别为：beijing、chengdu、chongqing、guangzhou、shanghai、shenzhen、tianjin，其中某人在beijing和shanghai居住过，则可编码为[1, 0, 0, 0, 1, 0, 0]。

preprocessing模块提供了MultiLabelBinarizer类进行多标签二值化编码，以下给出了编码示例。


In [18]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
mlb = MultiLabelBinarizer()
Y = [['beijing', 'chengdu'], ['chengdu', 'tianjin'], ['chongqing'],
      ['guangzhou'], ['shanghai'], ['shenzhen'], ['tianjin']]
mlb.fit(Y)  # 训练标签数据
print('coding:', mlb.transform([['beijing','shanghai']]))  # 打印新标签编码结果

coding: [[1 0 0 0 1 0 0]]


（3）特征按序编码（OrdinalEncoder） 

前面介绍的LabelEncoder是对标签文本进行按序编码，类似的，preprocessing模块提供了OrdinalEncoder类，可对数据每一列文本特征分别进行按序编码。

例如，数据特征中，存在表示性别的列，分别有两个取值"male"和"female"，可分别转换为整数编码0和1。再如，存在表示居住地的列，分别有"shanghai", "beijing", "shenzhen"三个取值，可编码为0，1和2。示例如下：


In [19]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
X = [['male', 'beijing'], ['female', 'shenzhen'], ['male', 'shanghai']]
oe.fit(X)  # 进行训练
print("categories: ", oe.categories_)  # 打印类别
# 打印对新数据的编码结果
print("encode: ", oe.transform([['male','shenzhen'], ['female', 'beijing']]))

categories:  [array(['female', 'male'], dtype=object), array(['beijing', 'shanghai', 'shenzhen'], dtype=object)]
encode:  [[1. 2.]
 [0. 0.]]


（4）独热编码（OneHotEncoder）

独热编码在preprocessing模块的OneHotEncoder类中实现，其原理类似于多标签二值化编码，只不过独热编码只能用于数据特征的编码，不能用于标签编码。设数据的某列特征有N个可能的取值，则将该列特征变换为长度为N的二进制特征向量，其中只有一个位置上是1，其余位置都是0，这也是与多标签二值化编码不同的。例如表示性别的列有2个取值，用长度为2的向量来表达，第0位代表是否为female，第1位代表是否为male，那么"female"编码为[1,0]，"male"编码为[0,1]。独热编码在一定程度上可以理解为扩充了特征数量，例如性别本身是一个特征，经过独热编码后，就变成了是否为男、是否为女两个特征。但当特征属性值较多时，数据经过独热编码可能会变得非常稀疏。

以下示例将数据的性别和出生地用独热编码来实现。


In [20]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
X = [['male', 'beijing'], ['female', 'shanghai'], 
['male', 'shenzhen'], ['female', 'beijing']]
ohe.fit(X)
print("categories: ", ohe.categories_)  # 查看特征取值  
print("encode:", ohe.transform([['female','shenzhen']]).toarray()) # 输出特征编码

categories:  [array(['female', 'male'], dtype=object), array(['beijing', 'shanghai', 'shenzhen'], dtype=object)]
encode: [[1. 0. 0. 0. 1.]]


## 3.2.4 离散化

在数据预处理中，常常需要对连续型数值的特征处理为离散型特征，例如存款金额，可以为0，也可以是几万或几亿，直接用其数值作为特征往往过于分散，因此可以转换为离散特征，将其分为几个档次。这种转换称为离散化（Discretization），本质上离散化操作实现了将连续属性到名义属性的转换。
下面介绍Processing包提供的两种最常见的离散化方法。

（1）K-bins离散化（KBinsDiscretizer） 

K-bins离散化是指将连续数值型特征值排序后通过使用k个等宽的区间分箱后编码，使得原来连续型变量划分为分类变量。
KBinsDiscretizer类中包含3个重要的参数，其中n_bins参数表示每维特征的分箱个数，默认为5，会被运用到所有导入的特征上。encode参数表示编码方式，可取值为"onehot"、"ordinal"、"onehot-dense"，默认"onehot"。strategy参数表示分箱方式，可取值为"uniform"、"quantile"、"kmeans"，分别表示等宽分箱、等位分箱、按聚类分箱，默认"quantile"。

以下代码给出了K-bins离散化示例。


In [21]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
X = np.array([[-3, 5, 15],[0, 6, 14],[6, 3, 11]])
kbd = KBinsDiscretizer(n_bins=[3, 2, 2],encode='ordinal')  
kbd.fit(X)
print('Edge:', kbd.bin_edges_)  # 显示分箱边界
print('coding:', kbd.transform(X))  # 对训练数据的离散结果
print('coding:', kbd.transform([[-10,2,0],[-1,5.5,16]]))  # 对新数据的离散结果

Edge: [array([-3., -1.,  2.,  6.]) array([3., 5., 6.]) array([11., 14., 15.])]
coding: [[0. 1. 1.]
 [1. 1. 1.]
 [2. 0. 0.]]
coding: [[0. 0. 0.]
 [1. 1. 1.]]


（2）二值化（Binarizer） 

特征二值化是指通过设定阈值将连续型数值特征划分得到布尔值（0或1）的过程，大于阈值的特征值映射为1，小于或等于阈值的特征值映射为0。在计算机视觉中，针对灰度图像的二值化的作用就是将图像变为黑白图像，以便于进行图像分割、目标提取等操作。
Binarizer类提供了参数threshold，用于设置各个维度上的阈值。示例如下：

In [22]:
from sklearn.preprocessing import Binarizer
X = [[1.,-1.,2.],[2.,0.,0.],[0.,1.,-1.]]
binarizer = Binarizer(threshold=[1,0,1]).fit(X)
print(binarizer.transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
